Note: `cv2.imshow()` is used!

In [16]:
import cv2
import math
import numpy as np

In [17]:
class EuclideanDistTracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0
        # History of center points
        self.history_of_center_points = {}


    def update(self, objects_rect):
        r = lambda: np.random.randint(0,255)
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 25:
                    self.center_points[id] = (cx, cy)
                    self.history_of_center_points[id].append((cx, cy))
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                self.history_of_center_points[self.id_count] = [(r(),r(),r()), (cx, cy)]
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_history_of_center_points = {}
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            history = self.history_of_center_points[object_id]
            new_center_points[object_id] = center
            new_history_of_center_points[object_id] = history

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        self.history_of_center_points = new_history_of_center_points.copy()
        return self.history_of_center_points


Running cell below will show video, press "ESC" to quit

In [19]:
tracker = EuclideanDistTracker()

cap = cv2.VideoCapture("./cars.mp4")

size = (684, 360)
out = cv2.VideoWriter('cars_tracked.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)

object_detector = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=40)

while True:
    ret, frame = cap.read()
    
    if ret == False:
        print("Not found")
        break
    
    mask = object_detector.apply(frame)
    allContours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contours = []
    for i in range(len(allContours)):
        if hierarchy[0][i][3] == -1:
            contours.append(allContours[i])
    
    detections = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 80:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1)
            detections.append([x, y, w, h])
            
    history = list(tracker.update(detections).values())

    for i in range(len(history)):
        color = history[i][0]
        new_history = []
        for tpl in history[i][1::15]:
            new_history.append(list(tpl))
        cv2.polylines(frame, [np.array(new_history, np.int32)], False, color, 2)

    cv2.imshow("Frame", frame)
    
    if ret:
        vidout=cv2.resize(frame,size)
        out.write(vidout)

    # press Esc to quit
    key = cv2.waitKey(30)
    if key == 27:
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Not found
